In [1]:
import tensorflow as tf

# Uncomment to see where your variables get placed (see below)
# tf.debugging.set_log_device_placement(True)

In [5]:

### 1.2 变量

### 1.2.1 创建变量
#变量通过 tf.Variable 类进行创建和跟踪。tf.Variable 表示张量，对它执行运算可以改变其值。
#要创建变量，请提供一个初始值。tf.Variable 与初始值的 dtype 相同。

my_tensor = tf.constant([[1,2,3],[4,5,6]])
my_variable = tf.Variable(my_tensor)
my_variable

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [3]:
# Variables can be all kinds of types, just like tensors
bool_variable = tf.Variable([False, False, False, True])
bool_variable

<tf.Variable 'Variable:0' shape=(4,) dtype=bool, numpy=array([False, False, False,  True])>

In [4]:
complex_variable = tf.Variable([5 + 4j, 6 + 1j])
complex_variable

<tf.Variable 'Variable:0' shape=(2,) dtype=complex128, numpy=array([5.+4.j, 6.+1.j])>

In [6]:
##变量与张量的定义方式和操作行为都十分相似，实际上，它们都是 tf.Tensor 支持的一种数据结构。
#与张量类似，变量也有 dtype 和形状，并且可以导出至 NumPy。
print("Shape: ", my_variable.shape)
print("DType: ", my_variable.dtype)
print("As NumPy: ", my_variable.numpy())

Shape:  (2, 3)
DType:  <dtype: 'int32'>
As NumPy:  [[1 2 3]
 [4 5 6]]


In [7]:
##大部分张量运算在变量上也可以按预期运行，不过变量无法重构形状。
tf.convert_to_tensor(my_variable)

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [13]:
tf.math.argmax(my_variable)

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([1, 1, 1], dtype=int64)>

In [15]:
tf.reshape(my_variable,[3,2])

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4],
       [5, 6]])>

In [16]:
my_variable  #my_variable本身并没有改变。Copying and reshaping

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [20]:
##可以使用 tf.Variable.assign 重新分配张量。调用 assign（通常）不会分配新张量，而会重用现有张量的内存。
#用于更新 TensorFlow 变量的值。它将新的值赋给变量或其部分数值。
a = tf.Variable([2.0, 3.0])
a.assign([4,5])

<tf.Variable 'UnreadVariable' shape=(2,) dtype=float32, numpy=array([4., 5.], dtype=float32)>

In [21]:
a  #a的值已经被更新


<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([4., 5.], dtype=float32)>

In [22]:
##如果在运算中像使用张量一样使用变量，那么通常会对支持张量执行运算。
##从现有变量创建新变量会复制支持张量。两个变量不能共享同一内存空间。
a = tf.Variable([2.0, 3.0])
# Create b based on the value of a
b = tf.Variable(a)
a.assign([5, 6])

# a and b are different
print(a.numpy())
print(b.numpy())

# There are other versions of assign
print(a.assign_add([2,3]).numpy())  # [7. 9.] #用于将一个给定的增量值加到变量上，并将结果赋给该变量。
print(a.assign_sub([7,9]).numpy())  # [0. 0.]



[5. 6.]
[2. 3.]
[7. 9.]
[0. 0.]


In [27]:
### 1.2.2 生命周期、命名和监视
## 变量命名
#tf.Variable 实例与其他 Python 对象的生命周期相同。如果没有对变量的引用，则会自动将其解除分配。
#为了便于跟踪和调试，您还可以为变量命名。两个变量可以使用相同的名称。
# Create a and b; they will have the same name but will be backed by
# different tensors.
a = tf.Variable(my_tensor, name="Mark")
# A new variable with the same name, but different value
# Note that the scalar add is broadcast
b = tf.Variable(my_tensor + 1, name="Mark")

# These are elementwise-unequal, despite having the same name
print(my_variable,'\n')
print(a,'\n')
print(b,'\n')
print(a == b)

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])> 

<tf.Variable 'Mark:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])> 

<tf.Variable 'Mark:0' shape=(2, 3) dtype=int32, numpy=
array([[2, 3, 4],
       [5, 6, 7]])> 

tf.Tensor(
[[False False False]
 [False False False]], shape=(2, 3), dtype=bool)


In [29]:
##变量名唯一性
#保存和加载模型时会保留变量名。默认情况下，模型中的变量会自动获得唯一变量名，
#所以除非您希望自行命名，否则不必多此一举。

#虽然变量对微分很重要，但某些变量不需要进行微分。在创建时，通过将 trainable 设置为 False 
#可以关闭梯度。例如，训练计步器就是一个不需要梯度的变量。
step_counter = tf.Variable(1, trainable=False)
step_counter

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=1>

In [32]:
### 1.2.3 放置变量和张量
#为了提高性能，TensorFlow 会尝试将张量和变量放在与其 dtype 兼容的最快设备上。
#这意味着如果有 GPU，那么大部分变量都会放置在 GPU 上。
#不过，您可以对此进行重写。在此代码段中，即使存在可用的 GPU，我们也可以在 CPU 上放置一个浮点张量和一个变量。
with tf.device("CPU:0"):
  # Create some tensors
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
  c = tf.matmul(a, b)
  print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [33]:
#您可以将变量或张量的位置设置在一个设备上，然后在另一个设备上执行计算。
#但这样会产生延迟，因为需要在两个设备之间复制数据。
#不过，如果您有多个 GPU 工作进程，但希望变量只有一个副本，则可以这样做。
with tf.device('CPU:0'):
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.Variable([[1.0, 2.0, 3.0]])

with tf.device('GPU:0'):
  # Element-wise multiply
  k = a * b

print(k)

tf.Tensor(
[[ 1.  4.  9.]
 [ 4. 10. 18.]], shape=(2, 3), dtype=float32)


In [ ]:
### 总结

#有时候特征空间较大，无法用一张GPU完成参数更新，需要多行卡完成计算，
#则可以考虑把参数拆分放在不同卡上。例如用户id单独放一张卡做embedding。